In [1]:
#import connection services to google big query and connect
from google.cloud import bigquery
from google.oauth2 import service_account

service_path = "C:\\Users\\hauer\\OneDrive - The University of Montana\\MSBA\\BMKT 670 Applied Data Analytics\\Wedge Project\\"
service_file = 'hauer-ada-wedgeproject-2894572d63a8.json' # this is your authentication information  
gbq_proj_id = 'hauer-ada-wedgeproject'  # change this to your project_id
gbq_dataset_id = 'wedge_full' # and change this to your data set ID

private_key = service_path + service_file

# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [2]:
# connect to sqlite and create the WedgeTask3 database file
import sqlite3
db = sqlite3.connect("WedgeTask3.db")
cur = db.cursor()

In [3]:
# First Query of Wedge data in GBQ - Date, Hour, Sales, Transactions, and Items
query1 = (
    """SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(hour FROM datetime)) AS Hour,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge_full.transArchive_clean` 
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour"""
)

# And we execute queries with `client.query`
results1 = client.query(
    query1,
    location="US",
)

In [4]:
# output the first query results as a text file
with open('Sales_by_Date_by_Hour.txt', 'w') as outfile :
    for line in results1 :
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")

In [5]:
# input the results of the first query into the WedgeTask3 database as the table Sales_by_Date_by_Hour
input_file1 = "Sales_by_Date_by_Hour.txt"

db = sqlite3.connect("WedgeTask3.db") # connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Date_by_Hour''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Date_by_Hour (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Date by Hour text data into the established table in the database
with open(input_file1, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Date_by_Hour (Date, Hour, Sales, Transactions, Items)
        VALUES (?,?,?,?,?)''', line)
db.commit()

In [6]:
# Second Query of Wedge data in GBQ - Owner, Year, Month, Sales, Transactions, and Items
query2 = (
    """SELECT card_no As Owner,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge_full.transArchive_clean` 
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Owner, Year, Month
    ORDER BY Owner, Year, Month DESC"""
)

# And we execute queries with `client.query`
results2 = client.query(
    query2,
    location="US",
)

In [7]:
# output the second query results as a text file
with open('Sales_by_Owner_by_Date.txt', 'w') as outfile :
    for line in results2 :
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")

In [8]:
# input the results of the first query into the WedgeTask3 database as the table Sales_by_Owner_by_Date
input_file2 = "Sales_by_Owner_by_Date.txt"

db = sqlite3.connect("WedgeTask3.db") # connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Owner_by_Date''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Owner_by_Date (
    Owner INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Owner by Date text data into the established table in the database
with open(input_file2, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Owner_by_Date (Owner, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line)
db.commit()

In [9]:
# Third Query of Wedge data in GBQ - Product, Date, Sales, Transactions, and Items
query3 = (
    """SELECT Upc AS UPC,
    description AS Product,
    a.department AS Department,
    b.dept_name AS Dept_Name,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge_full.transArchive_clean` a
    LEFT OUTER JOIN `wedge_full.wedge_dprtmt_lookup` b
    ON a.department = b.department
    WHERE card_no != 3
    AND a.department != 0
    AND a.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY UPC, Product, Department, Dept_Name, Year, Month
    ORDER BY UPC, Product, Department, Dept_Name, Year, Month DESC"""
)

# And we execute queries with `client.query`
results3 = client.query(
    query3,
    location="US",
)

In [ ]:
# output the thrid query results as a text file
with open('Sales_by_Product_by_Date.txt', 'w') as outfile :
    for line in results3 :
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")

In [ ]:
# input the results of the first query into the WedgeTask3 database as the table Sales_by_Product_by_Date
input_file3 = "Sales_by_Product_by_Date.txt"

db = sqlite3.connect("WedgeTask3.db")# connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Product_by_Date''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Product_by_Date (
    UPC STRING,
    Description STRING,
    Department FLOAT,
    Dept_Name STRING,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Product by Date text data into the established table in the database
with open(input_file3, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Product_by_Date (UPC, Description, Department, Dept_Name, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?,?)''', line)
db.commit()